In [1]:
# # 邮件配置
# from utils.MailSenderHTML import MailSenderHTML
# mail_filename = 'config/mail.json'
# msh = MailSenderHTML(mail_filename)

In [2]:
# 每日回顾
import sys
# sys.path.append("../datasource/DailyReview")

sys.path.append("../../")

from EMData import EMData
from WSData import WSData
from CH_VIX import CH_VIX

from Nations import Nations
from Liquidity import Liquidity
from Leverage import Leverage
from Economic import Economic
from Direction import Direction
from Sentiment import Sentiment

from MacroViewData import MacroViewData


from datetime import datetime,timedelta
import warnings
warnings.filterwarnings("ignore")

In [3]:
def update_img_data_dict(img_data_dict,fig,name):
    filename = result_path + '{}.png'.format(name)
    fig.savefig(filename,bbox_inches = 'tight', dpi=70)
    file = open(filename, "rb")
    img_data = file.read()
    file.close()
    img_data_dict[name] = img_data
    
    return img_data_dict

def repalce_message(html_content,content,name,isEmail = True):
    if isEmail:
        html_content = html_content.replace('替换{}图片'.format(name),'<img src="cid:{0}" alt="{0}">'.format(name))
    else:
#         html_content = html_content.replace('替换{}图片'.format(name),'<img src="/home/jquser/0.定时推送/datasource/DailyReview/data/{0}.png" alt="{0}">'.format(name))
        html_content = html_content.replace('替换{}图片'.format(name),'<img src="{0}.png" alt="{0}">'.format(name))
    html_content = html_content.replace('替换{}表格'.format(name),content)
    
    return html_content



In [4]:
# data_path = './datasource/DailyReview/data/'
path = '../'
data_path = path + 'process/'
result_path = path + 'result/'

In [5]:
em = EMData(data_path)
try:
    # 中国PPI CPI PMI 北向资金
    em.save_data(typeStr = 'north_money')
    
    em.save_data(typeStr = 'ppi')
    em.save_data(typeStr = 'cpi')
    em.save_data(typeStr = 'pmi')
    
    
    # 美国PMI
    em.save_data(typeStr = 'us_manufacturing_pmi')
    em.save_data(typeStr = 'us_non_manufacturing_pmi')
    
    # 信贷
    em.save_data(typeStr = 'loan')
    
except Exception as e:
    # 访问异常的错误编号和详细信息
    print(e.args)
    print("获取数据出错：东财")

In [6]:
mvd = MacroViewData(data_path)

try:
    # 中美利差
    mvd.save_data('us_fed_rate') 
    mvd.save_data('cn_dr007')
    
    # 美国CPI
    mvd.save_data('us_cpi')
    
    # 美国国债利差：10年-2年
    mvd.save_data('us_bond_10Y-2Y')
except Exception as e:
    # 访问异常的错误编号和详细信息
    print(e.args)
    print("获取数据出错：macroview")

('randomId',)
获取数据出错：macroview


In [7]:
ws = WSData(data_path)

try:
    ws.save_data(typeStr = 'USDCNY', count = 30)
    ws.save_data(typeStr = 'USDJPY', count = 30)
    ws.save_data(typeStr = 'US10YR', count = 30)
    ws.save_data(typeStr = 'USCL', count = 30)
    ws.save_data(typeStr = 'CN10YR', count = 30)
except Exception as e:
    # 访问异常的错误编号和详细信息
    print(e.args)
    print("获取数据出错：华尔街见闻")

In [8]:
start_date = '2015-01-01'
end_date = datetime.now().strftime("%Y-%m-%d")

try:
    #上证50波幅指数
    VIX = CH_VIX('510050.XSHG',start_date,end_date,data_path)
    VIX_ser = VIX.GetCBOE_VIX()

    # 沪深300波幅指数
    VIX = CH_VIX('510300.XSHG',start_date,end_date,data_path)
    VIX_ser = VIX.GetCBOE_VIX()

    # 中证1000波幅指数
    VIX = CH_VIX('000852.XSHG',start_date,end_date,data_path)
    VIX_ser = VIX.GetCBOE_VIX()
except Exception as e:
    # 访问异常的错误编号和详细信息
    print(e.args)


In [9]:
# def main():
# 读取模板文件
filename = path + 'content.html'
with open(filename,'rb') as f:
    html_content = f.read().decode('UTF-8')
    msg_content = html_content

img_data_dict = {}  


In [10]:
# from IPython.display import HTML
# HTML(html_content)

In [11]:
# 1 国别资金流
na = Nations(data_path)

## 美元人民币
fig,content = na.show_exchange_data(10)
name = 'usdcny'
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)

## 中美利差
fig,content = na.show_interest_rate_data(10)
name = 'interest_rate'
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)


## 北向资金
fig,content = na.show_north_money_data(10)
name = 'north_money'
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)

In [12]:
# 2 流动性偏好
lq = Liquidity()

## 2.1 美元
### 美元指数
fig,content = lq.show_dollar_data(10)
name = 'DINIW'
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)

### 美日汇率
fig,content = lq.show_usdjpy_data(10)
name = 'usdjpy'
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)

### 美国十年国债收益率
fig,content = lq.show_us10yr_data(10)
name = 'us10yr'
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)

### 纳斯达克100指数
fig,content = lq.show_ndx_data(10)
name = 'NDX'
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)

### 美元计价大宗商品
fig,content = lq.show_uscl_data(10)
name = 'uscl'
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)

## 2.2 人民币
#### 十年国债收益率
fig,content = lq.show_cn10yr_data(10)
name = 'cn10yr'
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)

### 万得金仓100
fig,content = lq.show_wdjc100_data(10)
name = 'wdjc100'
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)


#### 融资融券余额
l = Leverage()
fig,content = l.show_data()
name = 'leverage'
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)

### 股票型公募基金仓位
fig,content = l.show_position_data()
name = 'fund_position'
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)

AttributeError: 'NoneType' object has no attribute 'attrs'

In [13]:
# 3 经济景气度
e = Economic(data_path)

## 3.1 美国
### US PMI
fig,content = e.show_us_pmi_data(12)
name = 'us_pmi'
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)

### US CPI
fig,content = e.show_us_cpi_data(12)
name = 'us_cpi'
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)

### 美国国债利差：10年-2年
fig,content = e.show_us_bond_diff_data(12)
name = 'us_bond_10Y-2Y'
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)


## 3.2 中国
## PMI
name = 'pmi'
fig,content = e.show_pmi_data(12)
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)
### 新增信贷
name = 'xzxd'
fig,content = e.show_loan_data(12)
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)
### PPI-CPI剪刀差
name = 'ppi-cpi'
fig,content = e.show_ppi_cpi_data(12)
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)


In [14]:
# 4 市场倾向性
## 4.1 信用风险
d = Direction()
#### 大市值-小市值
name = 'credit_risk'
fig,content = d.show_risk_data(5)
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)
### 十年国债期货
name = 'treasury_bonds'
fig,content = e.show_bonds_data(5)
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)

## 4.2 剩余流动性
### 小市值
name = 'liquidity'
fig,content = d.show_liquidity_data(5)
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)
### 沪深两市成交额
name = 'turnover'
fig,content = d.show_volume_data(5)
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)

## 4.3 均衡与偏离
### 大市值
name = 'balance'
fig,content = d.show_balance_data(5)
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)
### 市场主线
name = 'mainline'
fig,content = d.show_main_data(5)
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)



In [15]:
# 5 市场情绪
s = Sentiment(data_path)
# 5.1 A股
### 恐慌贪婪指数
name = 'panic'
fig,content = s.show_panic_data(5)
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)

### 上证50波幅指数
name = 'vix_50'
fig,content = s.show_ch_vix_data('510050',5)
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)
### 沪深300波幅指数
name = 'vix_300'
fig,content = s.show_ch_vix_data('510300',5)
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)
### 中证1000波幅指数
name = 'vix_852'
fig,content = s.show_ch_vix_data('000852',5)
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)




JSEvalException: Uncaught Error: Malformed UTF-8 data at undefined:479:20
Error: Malformed UTF-8 data
    at Object.stringify (<anonymous>:479:27)
    at WordArray.init.toString (<anonymous>:216:41)
    at my_decode (<anonymous>:6000:22)
    at <anonymous>:1:46
    at <anonymous>:1:11162

In [ ]:
## 5.2 港股
### 恒指波幅指数
name = 'vix_hsi'
fig,content = s.show_vhsi_data(5)
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)


In [ ]:
# 当前时间
datatime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
msg_content = msg_content.replace('当前时间',datatime)
html_content = html_content.replace('当前时间',datatime)

# 发送邮件
date = datetime.now().strftime("%Y-%m-%d")
subject = '每日回顾【{}】'.format(date)
# msh.sendMail(subject,message)
#     msh.sendMail(subject,msg_content,img_data_dict)

# 保存最新结果
with open(result_path + "html_content.html",'w',encoding='GBK') as f:
    f.write(html_content)

    # 保存最新结果
with open(result_path + "index.html",'w',encoding='UTF-8') as f:
    f.write(html_content)
# return html_content
# html_content = main()

In [ ]:
# github
# !git remote rm origin
# !git remote add origin git@github.com:zhiyizhilu/macro.git
!git add .
!git commit -m "update"

In [ ]:
print(datatime)

In [ ]:
url = 'https://www.joinquant.com/user/74461734912/files/1.%E5%AE%9A%E6%97%B6%E6%8E%A8%E9%80%81/datasource/DailyReview/data/result/html_content.html'
print(url)

In [ ]:
!git push -u origin master

In [ ]:
url = 'https://zhiyizhilu.github.io/macro/'
print(url)